```
Copyright 2023 DeepMind Technologies Limited.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```

# An example implementing the three key questions

March and Olsen (2011) posit that humans generally act as though they choose their actions by answering three key questions:

1. What kind of situation is this?
2. What kind of person am I?
3. What does a person such as I do in a situation such as this?

The agents used in this example implement these components. The premise of the simulation is that 4 friends are stuck at a snowed in pub. Also, it has just come to light that Alice stole Bob's car and crashed it.


## Init and import

In [ ]:
#!pip install git+https://github.com/google-deepmind/concordia.git

In [ ]:
# @title Imports

import collections
import concurrent.futures
import datetime

import numpy as np
import sentence_transformers

#from google.colab import widgets  # pytype: disable=import-error
from IPython import display

from concordia.agents import basic_agent
from concordia.components import agent as components
from concordia import components as generic_components
from concordia.associative_memory import associative_memory
from concordia.associative_memory import blank_memories
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.clocks import game_clock
from concordia.components import game_master as gm_components
from concordia.environment import game_master
from concordia.metrics import goal_achievement
from concordia.metrics import common_sense_morality
from concordia.metrics import opinion_of_others
from concordia.utils import html as html_lib
from concordia.utils import measurements as measurements_lib
from concordia.language_model import gpt_model
from concordia.language_model import gcloud_model
from concordia.language_model import ollama_model
from concordia.language_model import local_llama3_model
from concordia.utils import plotting


In [ ]:
# Setup sentence encoder
st_model = sentence_transformers.SentenceTransformer('../all-mpnet-base-v2')
embedder = lambda x: st_model.encode(x, show_progress_bar=False)


In [ ]:
# @title Language Model - pick your model and provide keys
CLOUD_PROJECT_ID = '' #@param {type: 'string'}
GPT_API_KEY = 'sk-QQ7S86D8zhdjif2D11Ff1f2d81C94dB183Ea6eB7DdF1973a' #@param {type: 'string'}
GPT_BASE_URL = 'https://pro.aiskt.com/v1' 
GPT_MODEL_NAME = 'gpt-4' #@param {type: 'string'}

USE_CLOUD = False #@param {type: 'boolean'}

# if USE_CLOUD:
#   model = gcloud_model.CloudLanguageModel(project_id= CLOUD_PROJECT_ID)
# else:
#   model = gpt_model.GptLanguageModel(api_key=GPT_API_KEY, base_url=GPT_BASE_URL, model_name=GPT_MODEL_NAME)

#model = ollama_model.OllamaLanguageModel('llama3:instruct')
model = local_llama3_model.Llama3LanguageModel('../Meta-Llama-3-8B-Instruct',system_message='You are a chatbot who may chat with the user or do tasks like role play according to the instructions.')
model.sample_text("Who are you?")
#model.sample_text("What is the capital of France? ")

## Configuring the generic knowledge of players and GM.

In [ ]:
#@title Make the clock
time_step = datetime.timedelta(minutes=20)
SETUP_TIME = datetime.datetime(hour=20, year=2024, month=10, day=1)

START_TIME = datetime.datetime(hour=18, year=2024, month=10, day=2)
clock = game_clock.MultiIntervalClock(
    start=SETUP_TIME,
    step_sizes=[time_step, datetime.timedelta(seconds=10)])


In [ ]:
#@title Importance models
importance_model = importance_function.AgentImportanceModel(model)
importance_model_gm = importance_function.ConstantImportanceModel()


In [ ]:
# @title Generic memories are memories that all players and GM share.

shared_memories = [
    ('Donald Trump is the former president and Joe Biden is the current president.'),
]

# The generic context will be used for the NPC context. It reflects general
# knowledge and is possessed by all characters.
shared_context = model.sample_text(
    'Summarize the following passage in a concise and insightful fashion:\n'
    + '\n'.join(shared_memories)
    + '\n'
    + 'Summary:'
)
print(shared_context)


## Configure and build the players

---



In [ ]:
blank_memory_factory = blank_memories.MemoryFactory(
    model=model,
    embedder=embedder,
    importance=importance_model.importance,
    clock_now=clock.now,
)

formative_memory_factory = formative_memories.FormativeMemoryFactory(
    model=model,
    shared_memories=shared_memories,
    blank_memory_factory_call=blank_memory_factory.make_blank_memory,
)

In [ ]:
#@title Creating character backgrounds, goals and traits. Modify to explore how it influences the outcomes
NUM_PLAYERS = 2

scenario_premise = [
    (
        'Donald Trump and Joe Biden are debating against each other on topics like gun violence, climate change, economy, LGBT issues, and relationship with CHINA.'
    ),
]
player_configs = [
    formative_memories.AgentConfig(
        name='Donald Trump',
        gender='male',
        goal='Donald Trump wants to be selected as US president again after four years of Joe Biden presidency.',
        context=shared_context,
        traits='responsibility: mid; aggression: high',
    ),
    formative_memories.AgentConfig(
        name='Joe Biden',
        gender='male',
        goal='Joe Biden want to win more seat in the congress for the Democracy Party.',
        context=shared_context,
        traits='responsibility: low; aggression: mid',
    ),
]

In [ ]:
def build_agent(agent_config,
                player_names: list[str],
                measurements: measurements_lib.Measurements | None = None):

  mem = formative_memory_factory.make_memories(agent_config)

  time = generic_components.report_function.ReportFunction(
    name='Current time',
    function=clock.current_time_interval_str,
  )

  current_obs = components.observation.Observation(
            agent_name=agent_config.name,
      clock_now=clock.now,
      memory=mem,
      timeframe=clock.get_step_size(),
      component_name='current observations',
  )
  summary_obs = components.observation.ObservationSummary(
      agent_name=agent_config.name,
      model=model,
      clock_now=clock.now,
      memory=mem,
      components=[current_obs],
      timeframe_delta_from=datetime.timedelta(hours=4),
      timeframe_delta_until=datetime.timedelta(hours=1),
      component_name='summary of observations',
  )

#   self_perception = components.self_perception.SelfPerception(
#       name=f'answer to what kind of person is {agent_config.name}',
#       model=model,
#       memory=mem,
#       agent_name=agent_config.name,
#       clock_now=clock.now,
#   )
  situation_perception = components.situation_perception.SituationPerception(
      name=(f'answer to what kind of situation is {agent_config.name} in ' +
            'right now'),
      model=model,
      memory=mem,
      agent_name=agent_config.name,
      components=[current_obs, summary_obs],
      clock_now=clock.now,
  )
#   person_by_situation = components.person_by_situation.PersonBySituation(
#       name=(f'answer to what would a person like {agent_config.name} do in a ' +
#             'situation like this'),
#       model=model,
#       memory=mem,
#       agent_name=agent_config.name,
#       clock_now=clock.now,
#       components=[self_perception, situation_perception],
#       verbose=True,
#   )

#   initial_goal_component = generic_components.constant.ConstantComponent(
#       state=agent_config.goal)

#   persona = generic_components.sequential.Sequential(
#       name='persona',
#       components=[
#           self_perception,
#           situation_perception,
#           person_by_situation,
#       ]
#   )

  goal_metric = goal_achievement.GoalAchievementMetric(
      model=model,
      player_name=agent_config.name,
      player_goal=agent_config.goal,
      clock=clock,
      name='Goal Achievement',
      measurements=measurements,
      channel='goal_achievement',
      verbose=False,
  )
  
# #   value_metric = goal_achievement.GoalAchievementMetric(
# #       model=model,
# #       player_name=agent_config.name,
# #       player_goal=agent_config.value,
# #       clock=clock,
# #       name='Goal Achievement',
# #       measurements=measurements,
# #       channel='goal_achievement',
# #       verbose=False,
# #   )
  
#   morality_metric = common_sense_morality.CommonSenseMoralityMetric(
#       model=model,
#       player_name=agent_config.name,
#       clock=clock,
#       name='Morality',
#       verbose=False,
#       measurements=measurements,
#       channel='common_sense_morality',
#   )
  agent = basic_agent.BasicAgent(
      model,
      mem,
      agent_name=agent_config.name,
      clock=clock,
      verbose=False,
      components=[
                  #persona,
                  time,
                  current_obs,
                  situation_perception,
                  goal_metric,
                  #morality_metric
                  ],
      update_interval = time_step,
      collect_data=True,
      )
  
  reputation_metric = opinion_of_others.OpinionOfOthersMetric(
      model=model,
      player_name=agent_config.name,
      player_names=player_names,
      context_fn=agent.state,
      clock=clock,
      name='Opinion',
      verbose=False,
      measurements=measurements,
      channel='opinion_of_others',
      question='What is {opining_player}\'s opinion of {of_player}?',
  )
  agent.add_component(reputation_metric)
  return agent


In [ ]:
player_configs = player_configs[:NUM_PLAYERS]
player_names = [player.name for player in player_configs][:NUM_PLAYERS]
measurements = measurements_lib.Measurements()

players = []


for np in range(NUM_PLAYERS):
  agent = build_agent(player_configs[np], player_names, measurements)
  players.append(agent)


## Build GM

In [ ]:
game_master_memory = associative_memory.AssociativeMemory(
   sentence_embedder=embedder,
   importance=importance_model_gm.importance,
   clock=clock.now)

In [ ]:
# @title Create components of the Game Master
player_names = [player.name for player in players]

snowed_in_construct = generic_components.constant.ConstantComponent(
    state=(
        'The debate is open towards global media and publics and addresses billions of attentions.'),
    name='Fact')
scenario_knowledge = generic_components.constant.ConstantComponent(
    state=' '.join(shared_memories),
    name='Background')

player_status = gm_components.player_status.PlayerStatus(
    clock_now=clock.now,
    model=model,
    memory=game_master_memory,
    player_names=player_names)


convo_externality = gm_components.conversation.Conversation(
    players=players,
    model=model,
    memory=game_master_memory,
    clock=clock,
    burner_memory_factory=blank_memory_factory,
    components=[player_status, snowed_in_construct],
    cap_nonplayer_characters=3,
    shared_context=shared_context,
    verbose=False,
)

direct_effect_externality = gm_components.direct_effect.DirectEffect(
    players=players,
    model=model,
    memory=game_master_memory,
    clock_now=clock.now,
    verbose=False,
    components=[player_status]
)

relevant_events = gm_components.relevant_events.RelevantEvents(
    clock.now, model, game_master_memory)
time_display = gm_components.time_display.TimeDisplay(clock)


In [ ]:
# @title Create the game master object
env = game_master.GameMaster(
    model=model,
    memory=game_master_memory,
    clock=clock,
    players=players,
    components=[
        snowed_in_construct,
        scenario_knowledge,
        player_status,
        convo_externality,
        direct_effect_externality,
        relevant_events,
        time_display,
    ],
    randomise_initiative=True,
    player_observes_event=False,
    verbose=True,
)

## The RUN

In [ ]:
clock.set(START_TIME)

In [ ]:
for premise in scenario_premise:
  game_master_memory.add(premise)
  for player in players:
    player.observe(premise)


In [ ]:
# @title Expect about 2-3 minutes per step.
episode_length = 5  # @param {type: 'integer'}
for _ in range(episode_length):
  env.step()


## Summary and analysis of the episode

In [ ]:
# @title Metrics plotting

# group_by = collections.defaultdict(lambda: 'player')
# group_by['opinion_of_others'] = 'of_player'

# tb = widgets.TabBar([channel for channel in measurements.available_channels()])
# for channel in measurements.available_channels():
#   with tb.output_to(channel):
#     plotting.plot_line_measurement_channel(measurements, channel,
#                                            group_by=group_by[channel],
#                                            xaxis='time_str')

## Save results

In [ ]:
# # @title Summarize the entire story.
# all_gm_memories = env._memory.retrieve_recent(k=10000, add_time=True)

# detailed_story = '\n'.join(all_gm_memories)
# print('len(detailed_story): ', len(detailed_story))
# # print(detailed_story)

# episode_summary = model.sample_text(
#     f'Sequence of events:\n{detailed_story}'+
#     '\nNarratively summarize the above temporally ordered ' +
#     'sequence of events. Write it as a news report. Summary:\n',
#     max_characters=3500, max_tokens=3500, terminators=())
# print(episode_summary)

In [ ]:
# # @title Summarise the perspective of each player
# player_logs = []
# player_log_names = []
# for player in players:
#   name = player.name
#   detailed_story = '\n'.join(player._memory.retrieve_recent(k=1000,
#                                                             add_time=True))
#   summary = ''
#   summary = model.sample_text(
#       f'Sequence of events that happened to {name}:\n{detailed_story}'
#       '\nWrite a short story that summarises these events.\n'
#       ,
#       max_characters=3500, max_tokens=3500, terminators=())

#   all_player_mem = player._memory.retrieve_recent(k=1000, add_time=True)
#   all_player_mem = ['Summary:', summary, 'Memories:'] + all_player_mem
#   player_html = html_lib.PythonObjectToHTMLConverter(all_player_mem).convert()
#   player_logs.append(player_html)
#   player_log_names.append(f'{name}')


#Build and display HTML log of the experiment

In [ ]:
# history_sources = [env, direct_effect_externality, convo_externality]
# histories_html = [
#     html_lib.PythonObjectToHTMLConverter(history.get_history()).convert()
#     for history in history_sources]
# histories_names = [history.name() for history in history_sources]

In [ ]:
# gm_mem_html = html_lib.PythonObjectToHTMLConverter(all_gm_memories).convert()

# tabbed_html = html_lib.combine_html_pages(
#     histories_html + [gm_mem_html] + player_logs,
#     histories_names + ['GM'] + player_log_names,
#     summary=episode_summary,
#     title='Friends in a pub experiment',
# )

# tabbed_html = html_lib.finalise_html(tabbed_html)

In [ ]:
# display.HTML(tabbed_html)

#Interact with a specific player

In [ ]:
# sim_to_interact = 'Alice'  # @param ['Alice', 'Bob','Charlie', 'Dorothy', 'Ellen'] {type:"string"}
# user_identity = 'a close friend'  # @param {type:"string"}
# interaction_premise = f'{sim_to_interact} is talking to {user_identity}\n'  # @param {type:"string"}

# player_names = [player.name for player in players]
# player_by_name = {player.name: player for player in players}
# selected_player = player_by_name[sim_to_interact]
# interrogation = interaction_premise

In [ ]:
# utterence_from_user = 'Did Bob accept your apology?'  # @param {type:"string"}

# interrogation += f'{user_identity}: {utterence_from_user}'
# player_says = selected_player.say(interrogation)
# interrogation += f'\n{sim_to_interact}: {player_says}\n'
# print(interrogation)